In [7]:
import datetime as dt
import requests
import random
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import warnings

# Data Extraction from Malaysian ExternalTrade Statistics Online

In [11]:
# URL to the data extraction site
url = "https://metsonline.dosm.gov.my/tradev2/product-coderesult"

# Obtain csrf_token from the website
csrf_token = 'dG5PZ09oay0nPiIMATBTeTsZHFAgKSJdRQQVKwRYJBQADRckHStGYA=='

# Create headers for all posts
headers = {
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
"Cookie": "_ga_F8QG3XBL63=GS1.1.1679998862.1.1.1679998930.0.0.0; _ga_7Q47WMYPD5=GS1.1.1679998865.1.1.1679998930.0.0.0; _ga_HK905W412V=GS1.1.1679998865.1.1.1679998930.0.0.0; mp_f55a9b6a137ab21675ef8724f7864bde_mixpanel=%7B%22distinct_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24device_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%2C%22__timers%22%3A%20%7B%7D%7D; _ga_TF34V4E8EM=GS1.1.1679999831.1.1.1680004133.0.0.0; _ga_XSB7QCQ0M9=GS1.1.1681551362.3.1.1681551368.0.0.0; _ga=GA1.1.324035783.1679998837; PHPSESSID=3oi1at9dfj4ugn6nno2bsqdd60; _csrf=193a3c282709e8f8d32929d94372b30d2cc2a5402306edc5e12c04ca098a1819a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22SPmkNX8TOwS7oAIp1jZLK0O9tcXCRC-M%22%3B%7D; BNES_PHPSESSID=fk0GuBGEQJmmIMfAn9TJ6EK+FW/oxjgTr2AAO+vNTw1JHh1+YzlOkigmNmbMOtjxuyDVXvBqWx9L6ZHOVdyK0Unb9r8ZsuhgKRw0f3zlmA8=; BNES__csrf=T1JRVt/fspwmarDUSua9ANdq7/OTqhlhu8lprfzaWWaQtpaDzljVNyJUL1Y65guWawDI8LDmCHrD6nlIXud7o3ytcCBG5TDvi4Kd94SBmSCx1ED1HWM0YV+ztPXAQ5AhjCkcJExisSE5tRQSObfAv0WAFW/4frqxEg4pTyBtkcJZTi4Amki6jSChfqJAyShFuYOrg/ZKI0IsB8ShUR5UFcxqa3a28w/OAICYVhchalmoAAhmCMY22tozvta3gpBJGC+pONVCkNPozWC+7XVzDC5ZciTE34Uq"
}

In [3]:
# To select targeted year for data extraction
starting_year = 2000
current_year = dt.date.today().year

# Specify the year interval
year_interval = 23

targeted_year = []

for year in range (starting_year, current_year + 1, year_interval):
    if year + year_interval == current_year:
        ending_year = current_year
        targeted_year.append(ending_year)
        break
    elif year + year_interval < current_year:
        ending_year = year + year_interval - 1
        targeted_year.append(ending_year)
    elif year + year_interval > current_year:
        ending_year = current_year
        targeted_year.append(ending_year)

print(targeted_year)

[2023]


In [4]:
# Create empty list to store raw data
data_raw = []

max_retries = 2

start_time = time.time()
for year in targeted_year:
    payload_month = {
    '_csrf': csrf_token,
    'Tradev2[typeofsearch]': 'classification',
    'Tradev2[typedigit]': 7,
    'Tradev2[rangecode1]': 0,
    'Tradev2[rangecode2]': 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    'Tradev2[tradeflow][]': 'exports',
    # 'Tradev2[timeframe]': ,
    'Tradev2[timeframe]': 'month',
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    'Tradev2[mothdata]': starting_year,
    'Tradev2[mothdata2]': year,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    'Tradev2[geogroup]': 1,
    'Tradev2[geogroup]': 29,
    'Tradev2[codeshowby]': 'code'
    }
    
    for retry in range(max_retries):
        retry_delay = random.randint(10,15)  # seconds
        try:
            # Start requesting raw data from the targeted url
            monthly_raw = requests.post(url, data=payload_month, headers=headers)
            status = monthly_raw.status_code

            # Check the status_code of the request
            if monthly_raw.status_code == 200:
                data_raw.append(monthly_raw)
                print(f"Data Extraction for {starting_year}-{year} is Successful.")
            
            else:
                # Retry the data extraction if status_code != 200
                while status != 200:
                    print(f"Error: Response [{status}]. Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                    monthly_raw = requests.post(url, data=payload_month, headers=headers)
                    i = 0
                    i += 1
                    if i > max_retries: # Break the loop if maximum retries have reached
                        print(f"Fail to Retreive Data for {starting_year}-{year}.")
                        break
                    status = monthly_raw.status_code
                    
                data_raw.append(monthly_raw)
                print(f"Data Extraction for {starting_year}-{year} is Successful.")
                time.sleep(5)
                
            break # Break if successful
        
        except (requests.ConnectionError) as e:
            print(f"Error: {e}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Stop retrying if a different error occurs
        
    
    if year < current_year:
        starting_year = year + 1

print(data_raw)

end_time = time.time()
print(f'{end_time - start_time}')

Data Extraction for 2000-2023 is Successful.
[<Response [200]>]
49.422094106674194


In [5]:
# Create an empty list for storing the raw text from data extraction
data_extracted = []

# To extract only the table value
for raw in data_raw:
    individual_data = []
    result = BeautifulSoup(raw.text, 'html.parser') # Parse the HTML
    table = result.find('table', class_='table-bordered') # Look up for the table
    # Extract table rows
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all(['th', 'td'])
        cols = [col.get_text(strip=True) for col in cols]
        if not cols:
            next # Ignore improper row structure from adding into the list
        else:
            individual_data.append(cols)
    data_extracted.append(individual_data)

In [6]:
start_time = time.time()

# Select a subset of columns from the first row as column names
raw_df = pd.DataFrame()

for item in data_extracted:
    df = pd.DataFrame(item)
    for index, row in df.iterrows():
        row_data = list(row)
        if "GRAND TOTAL" in row_data:
            i = index
    df.drop(df.index[i], inplace=True)
    raw_df = pd.concat([raw_df, df], axis=1, ignore_index=True)
    
raw_df = raw_df.loc[:, ~raw_df.columns.duplicated()]
display(raw_df)

end_time = time.time()
print(f'{end_time-start_time}')

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,BIL,SITC,PRODUCT DESCRIPTION,Jan 2000,Feb 2000,Mar 2000,Apr 2000,May 2000,Jun 2000,Jul 2000,...,Dec 2022,Jan-Dec 2022,Jan 2023,Feb 2023,Mar 2023,Apr 2023,May 2023,Jun 2023,Jul 2023,Jan-Jul 2023
1,1,0,FOOD,"491,864,874","425,641,394","553,169,207","509,671,983","553,327,793","574,066,614","551,376,941",...,"4,044,772,865","44,610,210,833","3,079,999,429","3,539,279,631","4,044,164,918","3,251,410,911","3,898,340,823","3,853,964,611","3,847,711,463","25,514,871,786"
2,2,1,BEVERAGES AND TOBACCO,"92,332,416","77,214,349","100,950,924","98,515,223","111,185,482","112,047,681","88,254,854",...,"289,704,132","2,756,650,246","238,257,518","222,258,336","326,561,786","229,753,325","270,752,156","241,920,709","246,541,492","1,776,045,322"
3,3,2,"CRUDE MATERIALS, INEDIBLE","863,994,681","689,603,279","901,427,068","897,853,450","909,523,666","923,797,559","968,187,353",...,"2,508,340,672","31,262,920,541","2,399,620,561","2,346,324,739","2,715,999,243","1,946,644,340","2,454,013,437","2,628,562,391","2,750,910,622","17,242,075,333"
4,4,3,"MINERAL FUELS, LUBRICANTS, ETC.","2,843,136,683","2,881,747,034","3,188,319,069","2,455,441,551","2,651,869,264","2,885,169,740","3,117,527,228",...,"20,012,839,420","264,757,937,327","20,908,748,621","20,323,875,331","21,429,738,762","16,984,271,124","21,965,803,909","18,254,748,708","15,690,991,412","135,558,177,867"
5,5,4,ANIMAL AND VEGETABLE OILS AND FATS,"1,116,946,684","1,099,756,943","1,254,551,970","1,116,557,420","1,008,684,467","1,100,698,246","1,032,325,817",...,"9,375,034,566","100,658,316,648","5,597,732,713","5,629,973,235","7,647,033,259","5,828,048,670","5,767,124,163","5,696,471,751","5,840,106,478","42,006,490,269"
6,6,5,CHEMICALS,"880,219,925","1,072,302,224","1,086,130,053","1,087,280,719","1,231,347,129","1,376,535,977","1,418,732,775",...,"10,004,120,440","125,677,068,690","8,476,075,452","9,036,890,095","9,950,967,381","7,583,766,017","9,214,928,157","8,328,581,144","8,660,909,643","61,252,117,889"
7,7,6,MANUFACTURED GOODS,"1,927,880,545","1,910,675,351","2,133,213,423","2,174,508,072","2,291,918,346","2,191,009,429","2,236,784,776",...,"10,861,030,485","138,745,710,010","8,705,481,844","10,081,396,919","11,464,714,044","10,194,652,518","10,424,167,008","11,573,138,564","10,736,008,753","73,179,559,650"
8,8,7,MACHINERY & TRANSPORT EQUIPMENT,"14,958,386,186","16,986,581,083","20,139,482,974","17,989,050,119","18,776,956,887","19,830,751,054","20,301,875,068",...,"60,241,396,203","672,352,328,155","52,272,605,677","49,850,907,614","57,828,441,926","49,009,132,271","52,998,110,557","60,365,867,502","56,197,583,255","378,522,648,802"
9,9,8,MISCELLANEOUS MANUFACTURED ARTICLES,"2,026,711,654","1,922,449,106","2,368,704,586","2,333,513,182","2,480,702,004","2,505,145,689","2,722,661,037",...,"13,623,197,307","162,735,124,065","10,412,921,901","10,783,053,535","13,401,993,529","9,527,059,374","11,859,102,111","12,249,123,233","12,242,805,660","80,476,059,343"


0.030673503875732422


In [7]:
# Create an empty dataframe
data = pd.DataFrame()

for item in divided_list:
    df = pd.DataFrame(item)
    df = df.drop(list(range(0,2)), axis =1)
    df = pd.DataFrame.transpose(df) # Tranpose the dataframe
    last_column_index = df.columns[-1] # Remove the last column
    # Rename the columns name
    df = df.drop(columns=last_column_index)
    columns_label = df.iloc[0] 
    columns_label = columns_label.to_list()
    columns_label[0] = 'YEAR'
    df = df.reset_index(drop=True)
    df.columns = columns_label
    df = df.drop(index=[0], axis=0)
    
    # To concatanete dataframes into one
    data = pd.concat([data, df], ignore_index=True)

display(data.head(13))

NameError: name 'divided_list' is not defined

In [ ]:
# Data Cleaning Part
# Remove rows that contain the total value of a year
clean_data = data[data['YEAR'].apply(lambda x: '-' not in x)]

column_names = clean_data.columns.to_list()
del column_names[0]

# Remove the " , " from the number and convert it as integer.
for col in column_names:
    clean_data[col] = clean_data[col].astype(str).str.replace(',','').astype(np.int64)
    
display(clean_data.head())
print(clean_data.dtypes)

In [ ]:
# Save the data into .csv file
clean_data.to_csv('.\export-data-malaysia.csv', index=False)

# Data Extraction from World Bank Group (API)

In [4]:
import wbgapi as wb

In [ ]:
start_year = 2000
current_year = dt.date.today().year

# Extract Malaysia Total Population
pop_my = wb.data.DataFrame('SP.POP.TOTL','MYS', time=range(start_year,current_year+1)).transpose().reset_index()

# Extract Malaysia GDP
gdp_my = wb.data.DataFrame('NY.GDP.MKTP.CD', 'MYS', time=range(start_year,current_year+1)).transpose().reset_index()

# Extract Exchange Rate from USD to MYR
er_my = wb.data.DataFrame('PA.NUS.FCRF', 'MYS', time=range(start_year,current_year+1)).transpose().reset_index()

# Extract Malaysia Real Effective Exchagne Rate
reer_my = wb.data.DataFrame('PX.REX.REER', 'MYS', time=range(start_year,current_year+1)).transpose().reset_index()

In [ ]:
# Columns Renaming
col_labels = ['YEAR', 'POPULATION']
pop_my.columns = col_labels

col_labels = ['YEAR', 'GDP']
gdp_my.columns = col_labels

col_labels = ['YEAR', 'EXCHANGE RATE']
er_my.columns = col_labels

col_labels = ['YEAR', 'REER']
reer_my.columns = col_labels

In [1]:
# Merge the dataframe into one unified dataframe
merged_df =pd.merge(pop_my, gdp_my, on='YEAR', how='inner')
merged_df =pd.merge(merged_df, er_my, on='YEAR', how='inner')
merged_df =pd.merge(merged_df, reer_my, on='YEAR', how='inner')
display(merged_df.head())

merged_df.to_csv('.\wbg-data-malaysia.csv', index=False)

NameError: name 'pd' is not defined

In [2]:
import wbgapi as wb

In [33]:
wb.series.info(q = 'GDP')

id,value
EG.GDP.PUSE.KO.PP,GDP per unit of energy use (PPP $ per kg of oil equivalent)
EG.GDP.PUSE.KO.PP.KD,GDP per unit of energy use (constant 2017 PPP $ per kg of oil equivalent)
EG.USE.COMM.GD.PP.KD,"Energy use (kg of oil equivalent) per $1,000 GDP (constant 2017 PPP)"
NY.GDP.DEFL.KD.ZG,"Inflation, GDP deflator (annual %)"
NY.GDP.DEFL.KD.ZG.AD,"Inflation, GDP deflator: linked series (annual %)"
NY.GDP.DEFL.ZS,GDP deflator (base year varies by country)
NY.GDP.DEFL.ZS.AD,GDP deflator: linked series (base year varies by country)
NY.GDP.DISC.CN,Discrepancy in expenditure estimate of GDP (current LCU)
NY.GDP.DISC.KN,Discrepancy in expenditure estimate of GDP (constant LCU)
NY.GDP.MKTP.CD,GDP (current US$)


In [7]:
raw_data = wb.data.DataFrame(['NY.GDP.MKTP.KN', 'SP.POP.TOTL', 'PA.NUS.FCRF', 'PX.REX.REER'], 'MYS', timeColumns=True, numericTimeKeys=True).transpose().reset_index()

In [9]:
col_names = ['YEAR', 'GDP', 'EXCHANGE RATE', 'REER', 'POPULATION']
raw_data.columns = col_names
raw_data

,YEAR,GDP,EXCHANGE RATE,REER,POPULATION
0,1960,3.934806e+10,3.061220,NaN,7833782.0
1,1961,4.233772e+10,3.061220,NaN,8074805.0
2,1962,4.505624e+10,3.061220,NaN,8324218.0
3,1963,4.836283e+10,3.061220,NaN,8579350.0
4,1964,5.095458e+10,3.061220,NaN,8835925.0
...,...,...,...,...,...
58,2018,1.363766e+12,4.035130,88.662084,32399271.0
59,2019,1.423952e+12,4.142470,87.487217,32804020.0
60,2020,1.345144e+12,4.203482,84.400398,33199993.0
61,2021,1.386738e+12,4.143298,83.276834,33573874.0


In [14]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

In [24]:
# URL to the data extraction site
url = "https://metsonline.dosm.gov.my/tradev2/product-coderesult"

# Obtain csrf_token from the website
csrf_token = 'dG5PZ09oay0nPiIMATBTeTsZHFAgKSJdRQQVKwRYJBQADRckHStGYA=='

# Create headers for all posts
headers = {
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
"Cookie": "_ga_F8QG3XBL63=GS1.1.1679998862.1.1.1679998930.0.0.0; _ga_7Q47WMYPD5=GS1.1.1679998865.1.1.1679998930.0.0.0; _ga_HK905W412V=GS1.1.1679998865.1.1.1679998930.0.0.0; mp_f55a9b6a137ab21675ef8724f7864bde_mixpanel=%7B%22distinct_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24device_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%2C%22__timers%22%3A%20%7B%7D%7D; _ga_TF34V4E8EM=GS1.1.1679999831.1.1.1680004133.0.0.0; _ga_XSB7QCQ0M9=GS1.1.1681551362.3.1.1681551368.0.0.0; _ga=GA1.1.324035783.1679998837; PHPSESSID=3oi1at9dfj4ugn6nno2bsqdd60; _csrf=193a3c282709e8f8d32929d94372b30d2cc2a5402306edc5e12c04ca098a1819a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22SPmkNX8TOwS7oAIp1jZLK0O9tcXCRC-M%22%3B%7D; BNES_PHPSESSID=fk0GuBGEQJmmIMfAn9TJ6EK+FW/oxjgTr2AAO+vNTw1JHh1+YzlOkigmNmbMOtjxuyDVXvBqWx9L6ZHOVdyK0Unb9r8ZsuhgKRw0f3zlmA8=; BNES__csrf=T1JRVt/fspwmarDUSua9ANdq7/OTqhlhu8lprfzaWWaQtpaDzljVNyJUL1Y65guWawDI8LDmCHrD6nlIXud7o3ytcCBG5TDvi4Kd94SBmSCx1ED1HWM0YV+ztPXAQ5AhjCkcJExisSE5tRQSObfAv0WAFW/4frqxEg4pTyBtkcJZTi4Amki6jSChfqJAyShFuYOrg/ZKI0IsB8ShUR5UFcxqa3a28w/OAICYVhchalmoAAhmCMY22tozvta3gpBJGC+pONVCkNPozWC+7XVzDC5ZciTE34Uq"
}

In [29]:
def export_extract(url, start_year, end_year, max_retries):
    
    payload_month = {
    '_csrf': csrf_token,
    'Tradev2[typeofsearch]': 'classification',
    'Tradev2[typedigit]': 7,
    'Tradev2[rangecode1]': 0,
    'Tradev2[rangecode2]': 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    'Tradev2[tradeflow][]': 'exports',
    # 'Tradev2[timeframe]': ,
    'Tradev2[timeframe]': 'month',
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    'Tradev2[mothdata]': start_year,
    'Tradev2[mothdata2]': end_year,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    'Tradev2[geogroup]': 1,
    'Tradev2[geogroup]': 29,
    'Tradev2[codeshowby]': 'code'
    }
    
    # Define the retry strategy
    retry_strategy = Retry(
        total = max_retries,
        backoff_factor = 1.5,
        status_forcelist = [429, 500, 502, 503, 504]
    )
    
    # Create an HTTP adapter with the retry strategy and mount it to session
    adapter = HTTPAdapter(max_retries=retry_strategy)
    
    # Create a new session object
    session = requests.Session()
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    # Make a request using the session object
    response = session.post(url, data=payload_month, headers=headers)    
    
    if response.status_code == 200:
        print(f"SUCCESS: Data for year {end_year} has been extracted.")
    else:
        print(f"FAILED: Data for year {end_year} not able to be extracted.")
    
    return response.text

In [30]:
export_extract(url, 1997, 1997, 2)

SUCCESS: Data for year 1997 has been extracted.


'\n\n\n\n\n\n<br>\n<!-- download excell -->\n\n<div class="panel panel-default">\n  <div class="panel-heading"><h4>Please click here to download full data</h4></div>\n  <div class="panel-body">\n  \t  \t<div class="progress hideprogress">\n  <div id="progressbar" class="progressbar progress-bar progress-bar-warning  progress-bar-striped hideprogress" role="progressbar" aria-valuenow="0"\n  aria-valuemin="0" aria-valuemax="100" style="width:1%">\n  <center>\n   <span class=\'progresvalue\'></span>%\n     </center>\n  </div>\n</div>\n  \t<div class="btn-group">\n\n<button class=\'btn btn-pdf btndown\'  data-type=\'code\' data-url=\'downloadexcelltrade\' data-typeformat=\'excell\'>EXCEL</button>\n<button class=\'btn btn-success btndown\'  data-type=\'code\'  data-url=\'csvtrade\'  data-typeformat=\'csv\'>CSV</button>\n<span class=\'linkdown\'></span>\n\n\n\n\n\n</div>\n  </div>\n</div>\n\n\n<!-- end download excell -->\n\n\n\n<div style="overflow-y: scroll; height:500px !important;overflo

NameError: name 'response' is not defined

In [31]:
import logging

In [35]:
logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')

ERROR:root:This is an error message
CRITICAL:root:This is a critical message
